In [2]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
%matplotlib inline
np.random.seed(42)

In [ ]:
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
(x, y), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 40% del dataset para validación
validation = 0.40
N_validation_split = int(x.shape[0]*(1-validation))
print(N_validation_split)

# Training Set
x_train = x[:N_validation_split]
y_train = y[:N_validation_split]

# Cross Validation Set
x_test = x[N_validation_split:]
y_test = y[N_validation_split:]

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('Build model...')
model = Sequential()
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,embedding_dims,input_length=maxlen))
model.add(Dropout(0.2))
# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,kernel_size,padding='valid',activation='relu',strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())
# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,
validation_data=(x_test, y_test))

Loading data...
15000
15000 train sequences
10000 test sequences
Pad sequences (samples x time)
x_train shape: (15000, 400)
x_test shape: (10000, 400)
Build model...
Train on 15000 samples, validate on 10000 samples
Epoch 1/2
  384/15000 [..............................] - ETA: 112406s - loss: 0.6954 - acc: 0.4844

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("accuracy: ", score[1])

In [ ]:
weights = model.get_weights()[0]
biases = model.get_weights()[1]
print("Pesos :" , weights.shape)
print("Biases : " , biases)

In [ ]:
from keras.callbacks import ModelCheckpoint
from fnn_helper import PlotLosses
checkpointer = ModelCheckpoint(filepath='model_3.imdb.hdf5', verbose=1, save_best_only=True)
plot_losses = PlotLosses(plot_interval=1, evaluate_interval=None, x_val=x_train , y_val_categorical=y_train)

model.fit(x_train, y_train, batch_size=batch_size,epochs=epochs, validation_data=(x_test, y_test),callbacks=[plot_losses, checkpointer],)